In [1]:
import glob
import os
from os.path import join
import librosa
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [10]:
import tensorly as tl
from tensorly.decomposition import symmetric_parafac_power_iteration

In [2]:
# def cosine_distance(x, y):
#     return (1 - np.dot(x, y) / (np.linalg.norm(y) * np.linalg.norm(x))) * .5

def cosine_distance(x, y):
    return 1 - np.square(np.dot(x, y)) / (np.dot(x, x) * np.dot(y, y))


def get_features_filtered_by_label(features, labels, selected_label):
    return [feature for feature, label in zip(features, labels) if label == selected_label]

def get_k_main_eigenvectors(X, k=1):
    m1 = X.mean(axis=0)
    corr_matrix = np.cov(X.T)
    corr_eigens = np.linalg.eigvalsh(corr_matrix)
    m2 = corr_matrix + np.outer(m1, m1) - corr_eigens[0] * np.eye(corr_matrix.shape[0])
    w, v = np.linalg.eigh(m2)
    return v[:, -k:].T

def whiten(m2, eps=1e-18):
   # eigenvalue decomposition of the covariance matrix
   d, V = np.linalg.eigh(m2)

   # a fudge factor can be used so that eigenvectors associated with
   # small eigenvalues do not get overamplified.
   D = np.diag(1. / np.sqrt(d+eps))

   # whitening matrix
   W = np.dot(V, D)

   return W

def get_k_main_eigenvectors_cpd(X, k=1):
    m1 = X.mean(axis=0)
    N = X.shape[1]
    corr_matrix = np.cov(X.T)
    corr_eigens = np.linalg.eigvalsh(corr_matrix)
    m2 = corr_matrix + np.outer(m1, m1) - corr_eigens[0] * np.eye(corr_matrix.shape[0], dtype='float64')
    m3 = np.einsum('ij,ik,il->jkl',X,X,X) / N
    basis_sum = np.einsum('ij,ik,il->jkl',  m1[None, :], np.eye(N), np.eye(N)) + \
            np.einsum('ij,ik,il->jkl', np.eye(N),  m1[None, :], np.eye(N)) + \
            np.einsum('ij,ik,il->jkl', np.eye(N), np.eye(N),  m1[None, :])
    # basis_sum = np.einsum('j,ik,il->jkl',  m1, np.eye(N), np.eye(N)) + \
    #         np.einsum('ij,k,il->jkl', np.eye(N),  m1, np.eye(N)) + \
    #         np.einsum('ij,ik,l->jkl', np.eye(N), np.eye(N),  m1)
    m3 = m3 - corr_eigens[0] * basis_sum
    w = whiten(m2)
    # m3_whiten = tl.tucker_to_tensor((m3, [w, w, w]))
    m3_whiten = np.einsum('nmp,nk,ml,pd->kld', m3, w, w, w)
    lambdas, V = symmetric_parafac_power_iteration(m3_whiten, rank=k, n_iteration=20)
    A = np.linalg.pinv(w.T) @ V @ np.diag(lambdas)
    return A.T # V.T


def get_name(path):
    return os.path.splitext(os.path.split(path)[1])[0]

# --- test functions ---

def test(eigen_female, eigen_male, features_test, is_correct_function, debug=False, **kwargs):
    counter = 0
    num_eigenvectors = kwargs['num_eigenvectors']
    for feature in features_test:
        eigen_test = get_k_main_eigenvectors(feature, k=num_eigenvectors)
        dist_to_female = min([np.square(ete - etr).sum() for etr, ete in zip(eigen_female, eigen_test)])
        dist_to_male = min([np.square(ete - etr).sum() for etr, ete in zip(eigen_male, eigen_test)])
        correct = is_correct_function(dist_to_female, dist_to_male)
        if correct:
            counter += 1
    if debug:
        print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
    return counter / len(features_test)

def test_cosine(eigen_female, eigen_male, features_test, is_correct_function, debug=False, **kwargs):
    counter = 0
    for feature in features_test:
        dist_to_female = sum([min([cosine_distance(x, etr)
                                   for etr in eigen_female]) for x in feature])
        dist_to_male = sum([min([cosine_distance(x, etr)
                                   for etr in eigen_male]) for x in feature])
        correct = is_correct_function(dist_to_female, dist_to_male)
        if correct:
            counter += 1
    if debug:
        print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
    return counter / len(features_test)


# -----------------------------------------------------------------
# def normalize(x):
#     return x - x.mean()

def normalize(x):
    return x / np.linalg.norm(x, ord=2, axis=1, keepdims=True)

# check normalize
def get_k_main_eigenvectors_svd(features, k=1):
    _, _, vh = np.linalg.svd(normalize(features), full_matrices=False)
    return vh[:k]

#### CV for our approach:

In [3]:
def cross_validate(mfcc_list, labels_list, random_state_list, test_function, num_eigenvectors=1):
    results = {}
    accuracy_list = []
    accuracy_list_female = []
    accuracy_list_male = []
    for random_state in random_state_list:
        X_train, X_test, y_train, y_test = train_test_split(mfcc_list, labels_list, test_size=0.33,
                                                            random_state=random_state, stratify=labels_list)

        features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'F'))
        eigen_female = get_k_main_eigenvectors(features, k=num_eigenvectors)

        features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'M'))
        eigen_male = get_k_main_eigenvectors(features, k=num_eigenvectors)

        features_test = get_features_filtered_by_label(X_test, y_test, 'F')
        is_correct_function = lambda x, y: x < y
        accuracy_female = test_function(eigen_female, eigen_male, features_test, is_correct_function,
                                        num_eigenvectors=num_eigenvectors)

        features_test = get_features_filtered_by_label(X_test, y_test, 'M')
        is_correct_function = lambda x, y: x > y
        accuracy_male = test_function(eigen_female, eigen_male, features_test, is_correct_function,
                                     num_eigenvectors=num_eigenvectors)

#         print(f'f: {accuracy_female:.3f}, m: {accuracy_male:.3f}')
        accuracy_list.append((accuracy_female + accuracy_male) * .5)
        accuracy_list_female.append(accuracy_female)
        accuracy_list_male.append(accuracy_male)
        
    results['mean'] = np.mean(accuracy_list)
    results['mean_female'] = np.mean(accuracy_list_female)
    results['mean_male'] = np.mean(accuracy_list_male)
    
    results['std'] = np.std(accuracy_list)
    results['std_female'] = np.std(accuracy_list_female)
    results['std_male'] = np.std(accuracy_list_male)
    
    return results


#### CV for the article approach:

In [4]:
def cross_validate_article(mfcc_list, labels_list, random_state_list, test_function, num_eigenvectors=1):
    results = {}
    accuracy_list = []
    accuracy_list_female = []
    accuracy_list_male = []
    for random_state in random_state_list:
        X_train, X_test, y_train, y_test = train_test_split(mfcc_list, labels_list, test_size=0.33,
                                                            random_state=random_state, stratify=labels_list)

        features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'F'))
        eigen_female = get_k_main_eigenvectors_cpd(features, k=num_eigenvectors)

        features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'M'))
        eigen_male = get_k_main_eigenvectors_cpd(features, k=num_eigenvectors)

        features_test = get_features_filtered_by_label(X_test, y_test, 'F')
        is_correct_function = lambda x, y: x < y
        accuracy_female = test_function(eigen_female, eigen_male, features_test, is_correct_function,
                                        num_eigenvectors=num_eigenvectors)

        features_test = get_features_filtered_by_label(X_test, y_test, 'M')
        is_correct_function = lambda x, y: x > y
        accuracy_male = test_function(eigen_female, eigen_male, features_test, is_correct_function,
                                     num_eigenvectors=num_eigenvectors)

#         print(f'f: {accuracy_female:.3f}, m: {accuracy_male:.3f}')
        accuracy_list.append((accuracy_female + accuracy_male) * .5)
        accuracy_list_female.append(accuracy_female)
        accuracy_list_male.append(accuracy_male)
        
    results['mean'] = np.mean(accuracy_list)
    results['mean_female'] = np.mean(accuracy_list_female)
    results['mean_male'] = np.mean(accuracy_list_male)
    
    results['std'] = np.std(accuracy_list)
    results['std_female'] = np.std(accuracy_list_female)
    results['std_male'] = np.std(accuracy_list_male)
    
    return results

## Khanty dataset (309 F, 211 M, 520 total)

In [5]:
paths = sorted(glob.glob('khanty_4/*.wav'))

#### Experiments with different sizes of feature vectors

#### 26:

In [6]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=26).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 520/520 [02:49<00:00,  3.07it/s]


In [7]:
np.random.seed(122)
random_state_list = np.random.randint(0, 1000000, size=25)

#### Experiments with different number of eigenvectors (1, 2, 4):

### Our approach, 1 ev

In [8]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.829 std: 0.037
F mean: 0.802 std: 0.035
M mean: 0.856 std: 0.062


### Article approach, 1 ev

In [11]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.840 std: 0.025
F mean: 0.828 std: 0.040
M mean: 0.852 std: 0.039


### Our approach, 2 ev

In [12]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=2)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.829 std: 0.038
F mean: 0.801 std: 0.035
M mean: 0.856 std: 0.062


### Article approach, 2 ev

In [13]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=2)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.843 std: 0.022
F mean: 0.835 std: 0.038
M mean: 0.851 std: 0.038


### Our approach, 4 ev

In [14]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.829 std: 0.038
F mean: 0.801 std: 0.035
M mean: 0.856 std: 0.062


### Article approach, 4 ev

In [15]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.847 std: 0.030
F mean: 0.871 std: 0.044
M mean: 0.822 std: 0.080


#### 20:

In [16]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 520/520 [02:19<00:00,  3.73it/s]


In [17]:
np.random.seed(122)
random_state_list = np.random.randint(0, 1000000, size=19)

#### Experiments with different number of eigenvectors (1 and 4):

### Our approach, 1 ev

In [18]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.835 std: 0.038
F mean: 0.802 std: 0.041
M mean: 0.868 std: 0.052


### Article approach, 1 ev

In [19]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.825 std: 0.029
F mean: 0.810 std: 0.038
M mean: 0.841 std: 0.044


### Our approach, 4 ev

In [20]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.835 std: 0.038
F mean: 0.802 std: 0.041
M mean: 0.868 std: 0.051


### Article approach, 4 ev

In [21]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.825 std: 0.047
F mean: 0.929 std: 0.068
M mean: 0.720 std: 0.109


#### 13:

In [22]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 520/520 [02:29<00:00,  3.47it/s]


In [23]:
np.random.seed(122)
random_state_list = np.random.randint(0, 1000000, size=12)

#### Experiments with different number of eigenvectors (1 and 4):

### Our approach, 1 ev

In [24]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.81814 std: 0.037
F mean: 0.76961 std: 0.048
M mean: 0.86667 std: 0.041


### Article approach, 1 ev


In [25]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.79603 std: 0.023
F mean: 0.69444 std: 0.047
M mean: 0.89762 std: 0.036


### Our approach, 4 ev

In [26]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.81936 std: 0.037
F mean: 0.77206 std: 0.047
M mean: 0.86667 std: 0.041


### Article approach, 4 ev

In [27]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.73732 std: 0.051
F mean: 0.57108 std: 0.121
M mean: 0.90357 std: 0.040


#### For Khanty data the best performance of our approach (0.835) has been reached with 20-sized feature vectors and the best performance of the article approach (0.847) has been reached with 26-sized feature vectors and 4 eigenvectors. <br>
An accuracy for male and female audio data is almost equal except for the case of 13-sized feature vectors: for male samples an accuracy is much higher.

## TIMIT DR1 (New England dialect) corpus (180 F, 310 M, 490 total) : 

In [28]:
paths = sorted(glob.glob('TIMIT_DR1/*.wav'))

#### Experiments with different sizes of feature vectors 

#### 20:

In [36]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 490/490 [01:02<00:00,  7.85it/s]


In [ ]:
np.random.seed(122)
random_state_list = np.random.randint(0, 1000000, size=19)

#### Experiments with different number of eigenvectors (1 and 4):

### Our approach, 1 ev

In [39]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.84283 std: 0.023
F mean: 0.74912 std: 0.046
M mean: 0.93653 std: 0.022


### Article approach, 1 ev

In [38]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.90258 std: 0.021
F mean: 0.94035 std: 0.021
M mean: 0.86481 std: 0.031


### Our approach, 4 ev

In [40]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.84283 std: 0.023
F mean: 0.74912 std: 0.046
M mean: 0.93653 std: 0.022


### Article approach, 4 ev

In [41]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.87614 std: 0.023
F mean: 0.80439 std: 0.039
M mean: 0.94788 std: 0.023


#### 13:

In [42]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 490/490 [00:59<00:00,  8.25it/s]


In [43]:
np.random.seed(122)
random_state_list = np.random.randint(0, 1000000, size=12)

#### Experiments with different number of eigenvectors (1 and 4):

### Our approach, 1 ev

In [44]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.82361 std: 0.021
F mean: 0.71667 std: 0.039
M mean: 0.93056 std: 0.024


### Article approach, 1 ev

In [45]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.79052 std: 0.027
F mean: 0.98056 std: 0.016
M mean: 0.60049 std: 0.054


### Our approach, 4 ev

In [46]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.82361 std: 0.021
F mean: 0.71667 std: 0.039
M mean: 0.93056 std: 0.024


### Article approach, 4 ev

In [47]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.5f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.5f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.5f} std: {results['std_male']:.3f}")

AV mean: 0.83272 std: 0.026
F mean: 0.97917 std: 0.015
M mean: 0.68627 std: 0.049


#### For TIMIT data the best performance of our approach (0.84283) has been reached with 20-sized feature vectors and the best performance of the article approach (0.90258)  has been reached with 20-sized feature vectors and 1 etigenvector.  <br>
In case of our approach an accuracy for male audio data is higher than for female one. In case of the article approach, on the contrary, an accuracy for female audio data is higher than for male one.

## SHRUTI (the Bengali language) corpus (278 F, 261 M, 539 total)

In [52]:
paths = sorted(glob.glob('SHRUTI_small/*.wav'))

#### Experiments with different sizes of feature vectors 

#### 26:

In [54]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=26).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 539/539 [02:32<00:00,  3.54it/s]


In [55]:
np.random.seed(122)
random_state_list = np.random.randint(0, 1000000, size=25)

#### Experiments with different number of eigenvectors (1 and 4):


### Our approach, 1 ev

In [56]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.842 std: 0.036
F mean: 0.689 std: 0.072
M mean: 0.995 std: 0.007


### Article approach, 1 ev


In [57]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.971 std: 0.011
F mean: 0.989 std: 0.010
M mean: 0.953 std: 0.024


### Our approach, 4 ev


In [58]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.844 std: 0.036
F mean: 0.693 std: 0.072
M mean: 0.995 std: 0.007


### Article approach, 4 ev


In [59]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.931 std: 0.035
F mean: 0.989 std: 0.024
M mean: 0.873 std: 0.080


#### 20:

In [60]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 539/539 [02:17<00:00,  3.93it/s]


In [61]:
np.random.seed(122)
random_state_list = np.random.randint(0, 1000000, size=19)

#### Experiments with different number of eigenvectors (1 and 4):


### Our approach, 1 ev

In [62]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.807 std: 0.041
F mean: 0.616 std: 0.081
M mean: 0.997 std: 0.005


### Article approach, 1 ev

In [63]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.954 std: 0.014
F mean: 0.989 std: 0.010
M mean: 0.919 std: 0.031


### Our approach, 4 ev

In [64]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.809 std: 0.039
F mean: 0.622 std: 0.077
M mean: 0.997 std: 0.005


### Article approach, 4 ev

In [65]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.969 std: 0.013
F mean: 0.975 std: 0.026
M mean: 0.963 std: 0.023


#### 13:

In [66]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 539/539 [02:12<00:00,  4.08it/s]


In [67]:
np.random.seed(122)
random_state_list = np.random.randint(0, 1000000, size=12)

#### Experiments with different number of eigenvectors (1 and 4):


### Our approach, 1 ev

In [68]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.801 std: 0.036
F mean: 0.608 std: 0.074
M mean: 0.994 std: 0.008


### Article approach, 1 ev

In [69]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=1)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.884 std: 0.020
F mean: 0.992 std: 0.009
M mean: 0.777 std: 0.039


### Our approach, 4 ev

In [70]:
results = cross_validate(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.803 std: 0.033
F mean: 0.612 std: 0.068
M mean: 0.994 std: 0.008


### Article approach, 4 ev

In [71]:
results = cross_validate_article(mfcc_list, labels_list, random_state_list, test_function=test_cosine, num_eigenvectors=4)
print(f"AV mean: {results['mean']:.3f} std: {results['std']:.3f}")
print(f"F mean: {results['mean_female']:.3f} std: {results['std_female']:.3f}")
print(f"M mean: {results['mean_male']:.3f} std: {results['std_male']:.3f}")

AV mean: 0.779 std: 0.095
F mean: 0.997 std: 0.005
M mean: 0.560 std: 0.194


#### The best perfomance (0.844) for SHRUTI  corpus in our approach was reached with 26-sized feature vectors and 4 eigenvectors. In the article approach the best perfomance (0.971) was shown with 26-sized feature vectors and 1 eigenvector.  <br>
In our approach an accuracy for male audio data was much higher than for female one. In the article approach, on the contrary, female samples received higher accuracy.

#### All in all, the best average accuracy (0.971) was perfomed on SHRUTI dataset with the article approach using 26-sized feature vectors and 1 eigenvector.